# MLP

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

class MLP(nn.module):
    def __init__(self,input_size,hidden_size,num_classes):
        super(MLP,self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, num_classes)
        
    def forward(self,x):
        x = x.view(-1, self.input_size)
        x = torch.relu(self.fc1(x))
        '''
        或者这样写
        x = self.fc1(x)
        x = torch.relu(x)
        '''
        x = self.fc2(x)
        return x
    
# 1. 创建一个MLP实例
input_size = 576 # 假设每个输入有784个特征（例如，24x24像素的图像）
hidden_size = 512 # 第一个隐藏层的神经元数量
num_classes = 10 # 输出类别数量
mlp = MLP(input_size, hidden_size, num_classes)
criterion = nn.MSELoss()
optimizer = optim.SGD(mlp.parameters(), lr=1e-4)

x_train = torch.randn(64, input_size) 
y_train = torch.randn(64, num_classes)

for epoch in range(100):
    # 进行100个训练周期
    # 前向传播 
    outputs = mlp(x_train) 
    loss = criterion(outputs, y_train)

    # 反向传播和优化 
    optimizer.zero_grad()
    loss.backward() 
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/100], Loss: {loss.item():.4f}')

with torch.no_grad():
    sample_data = torch.randn(1, input_size) 
    predictions = mlp(sample_data) 
    print(predictions)
        
        


        

# Self-Attention

In [ ]:
import torch
import torch.nn as nn
import math

class SelfAttention(nn.module):
    def __init__(self, hidden_dim):
        super(SelfAttention, self).__init__()
        self.hidden_dim = hidden_dim
        self.q_proj = nn.Linear(hidden_dim, hidden_dim)
        self.k_proj = nn.Linear(hidden_dim, hidden_dim)
        self.v_proj = nn.Linear(hidden_dim, hidden_dim)
        
    def forward(self, x):
        q = self.q_proj(x)
        k = self.k_proj(x)
        v = self.v_proj(x)
        attn_weight = torch.matmul(q, k.transpose(-2, -1))
        attn_weight = torch.softmax(attn_weight/math.sqrt(self.hidden_dim), dim=-1)
        output = torch.matmul(attn_weight, v)
        return output

# MHA

In [ ]:
import torch
import torch.nn as nn

class MHA(nn.Module):
    def __init__(self, hidden_dim, num_heads,attention_dropout=0.1, output_dropout=0.1):
        super(MHA, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_heads = num_heads
        self.head_dim = hidden_dim / num_heads
        self.q_proj = nn.Linear(hidden_dim, hidden_dim)
        self.k_proj = nn.Linear(hidden_dim, hidden_dim)
        self.v_proj = nn.Linear(hidden_dim, hidden_dim)
        self.out_proj = nn.Linear(hidden_dim, hidden_dim)
        self.attn_dropout = nn.Dropout(attention_dropout)
        self.output_dropout = nn.Dropout(output_dropout)
        
    def forward(self, x):
        batch_size, seq_len, _ = x.shape
        # shape 变成 （batch_size, num_head, seq_len, head_dim）
        Q = self.q_proj(x).view(batch_size, seq_len, self.num_heads, self.head_dim).permute(#另一种写法
            0, 2, 1, 3
        )
        K = self.k_proj(x).view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        V = self.v_proj(x).view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        
        attn_weight = torch.softmax(Q @ K.transpose(-2, -1)/math.sqrt(self.head_dim),dim = -1)
        attn_weight = self.dropout(attn_weight)
        output = attn_weight @ V
        # contiguous重新分配内存，使张量在内存中是连续的，view函数要求输入的张量在内存中是连续的
        output = output.transpose(1,2).contiguous().view(batch_size, seq_len, self.hidden_dim)
        output = self.out_proj(output)
        output = self.output_dropout(output)
        
        return output
        
        
        
        

# 交叉熵+Softmax

In [1]:
import numpy as np
import torch
import torch.nn as nn

def mannul_softmax(x):
    """
    手动实现softmax函数
    :param x: 输入数组
    :return: softmax输出
    """
    # 防止溢出，减去最大值
    # torch.max(x, axis=1, keepdims=True)返回一个元组，第一个元素是最大值，第二个元素是最大值的索引
    # 使用values属性获取最大值,indices属性获取索引
    max_val = torch.max(x, axis=1, keepdims=True).values
    e_x = torch.exp(x - max_val)
    return e_x / torch.sum(e_x, dim=1, keepdims=True)

def cross_entropy_loss(y_true, y_pred):
    """
    计算交叉熵损失
    :param y_true: 真实标签，one-hot编码
    :param y_pred: 预测值，softmax输出
    :return: 交叉熵损失
    """
    # 防止log(0)的情况
    epsilon = 1e-15
    #数组 a 中所有小于 a_min 的值会被替换为 a_min。
    #数组 a 中所有大于 a_max 的值会被替换为 a_max。
    # y_pred = mannul_softmax(y_pred)
    y_pred = torch.clip(y_pred, epsilon, 1 - epsilon)
    
    # 计算交叉熵损失
    loss = -torch.sum(y_true * torch.log(y_pred), axis=1)
    
    return torch.mean(loss)


# 示例
# 1. 真实标签 (one-hot 编码)
y_true = torch.tensor([[0, 1, 0],
                       [1, 0, 0],
                       [0, 0, 1]], dtype=torch.float32) # 使用 float 类型以匹配计算

# 2. 模拟模型的原始输出 (logits - 未经 Softmax)
logits = torch.tensor([[0.5, 2.0, -1.0],  # 第一个样本，类别1得分最高
                       [3.0, 0.1, 0.1],  # 第二个样本，类别0得分最高
                       [-2.0, 0.5, 1.5]], # 第三个样本，类别2得分最高
                      dtype=torch.float32)

# 3. 使用手动实现的 Softmax 将 logits 转换为概率
y_pred_probs = mannul_softmax(logits)
print("Logits:\n", logits)
print("\nProbabilities (after manual_softmax):\n", y_pred_probs)

# 4. 使用手动实现的交叉熵损失函数计算损失
manual_loss = cross_entropy_loss(y_true, y_pred_probs)
print(f"\nManual Softmax + Manual Cross Entropy Loss: {manual_loss.item():.4f}")

# 5. (可选) 使用 PyTorch 内置函数进行验证
# 注意: nn.CrossEntropyLoss 结合了 Softmax 和 NLLLoss，它期望输入原始 logits 和类别索引
criterion_pytorch = nn.CrossEntropyLoss()
# 将 one-hot 真实标签转换为类别索引
y_true_indices = torch.argmax(y_true, dim=1)
pytorch_loss = criterion_pytorch(logits, y_true_indices)
print(f"PyTorch nn.CrossEntropyLoss (takes logits): {pytorch_loss.item():.4f}")

# 比较两个损失值，它们应该非常接近（可能因数值精度略有差异）

Logits:
 tensor([[ 0.5000,  2.0000, -1.0000],
        [ 3.0000,  0.1000,  0.1000],
        [-2.0000,  0.5000,  1.5000]])

Probabilities (after manual_softmax):
 tensor([[0.1753, 0.7856, 0.0391],
        [0.9009, 0.0496, 0.0496],
        [0.0216, 0.2631, 0.7153]])

Manual Softmax + Manual Cross Entropy Loss: 0.2269
PyTorch nn.CrossEntropyLoss (takes logits): 0.2269


# KL散度
**真实分布 (p)​**​  
  例：`[1, 0, 0]`（样本明确属于第一类）  
  信息熵（最小平均编码长度）：
  $$
  H(p) = -\sum_{i=1}^C p(x_i)\log p(x_i)
  $$

**拟合分布 (q)​**​  
  例：`[0.7, 0.2, 0.1]`（模型预测概率）  
  交叉熵（实际编码长度期望）：
  $$
  H(p,q) = -\sum_{i=1}^C p(x_i)\log q(x_i)
  $$

**KL散度**​  
  衡量分布差异（吉布斯不等式保证 $H(p,q) \geq H(p)$）：
  $$
  D_{KL}(p \| q) = H(p,q) - H(p) = \sum_{i=1}^C p(x_i)\log \frac{p(x_i)}{q(x_i)}
  $$

In [ ]:
def kl_divergence(y_true, y_pred, eps=1e-15):
    """
    计算 KL 散度，即 D_KL(p || q)
    
    参数:
        p: torch.Tensor, 真实概率分布 (每行代表一个样本的概率分布)
        q: torch.Tensor, 预测概率分布 (每行代表一个样本的概率分布)
        eps: float, 防止 log(0) 的值
        
    返回:
        平均 KL 散度
    """
    y_true = torch.clamp(y_true, eps, 1)
    y_pred = torch.clamp(y_pred, eps, 1)
    # 对每个样本，计算 sum(p * (log(p) - log(q)))
    kl = torch.sum(y_true * (torch.log(y_true) - torch.log(y_pred)), dim=1)
    return torch.mean(kl)
# 示例 1：使用已有的 y_true 和 y_pred_probs
# 计算结果中可以得知，在分类任务中，交叉熵损失等于KL散度
kl1 = kl_divergence(y_true, y_pred_probs)
print(f"Sample 1 - KL Divergence: {kl1.item():.4f}")

# 示例 2：自定义的概率分布
p2 = torch.tensor([[0.4, 0.4, 0.2]], dtype=torch.float32)
q2 = torch.tensor([[0.3, 0.5, 0.2]], dtype=torch.float32)
kl2 = kl_divergence(p2, q2)
print(f"Sample 2 - KL Divergence: {kl2.item():.4f}")

Sample 1 - KL Divergence: 0.2269
Sample 2 - KL Divergence: 0.0258
